                                                       Name: Yan Sun  
                                                       PID: A53240727

1.The code currently does not perform any train/test splits. Split the data into training, validation, and
test sets, via 1/3, 1/3, 1/3 splits. Use the first third, second third, and last third of the data (respectively).
After training on the training set, report the accuracy of the classier on the validation and test sets (1
mark).

In [2]:
import numpy
import urllib
import scipy.optimize
import random
from math import exp
from math import log

def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)

print("Reading data...")
data = list(parseData("file:beer_50000.json"))
print("done")

Reading data...
done


In [8]:
def feature(datum):
  feat = [1, datum['review/taste'], datum['review/appearance'], \
          datum['review/aroma'], datum['review/palate'], \
          datum['review/overall']]
  return feat

In [9]:
X = [feature(d) for d in data]
y = [d['beer/ABV'] >= 6.5 for d in data]

def inner(x,y):
  return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
  return 1.0 / (1 + exp(-x))

In [5]:
length = int(len(data)/3)

X_train = X[:length]
y_train = y[:length]

X_validation = X[length:2*length]
y_validation = y[length:2*length]

X_test = X[2*length:]
y_test = y[2*length:]

In [6]:
##################################################
# Logistic regression by gradient ascent         #
##################################################

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
  loglikelihood = 0
  for i in range(len(X)):
    logit = inner(X[i], theta)
    loglikelihood -= log(1 + exp(-logit))
    if not y[i]:
      loglikelihood -= logit
  for k in range(len(theta)):
    loglikelihood -= lam * theta[k]*theta[k]
  # for debugging
  # print("ll =" + str(loglikelihood))
  return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
  dl = [0]*len(theta)
  for i in range(len(X)):
    logit = inner(X[i], theta)
    for k in range(len(theta)):
      dl[k] += X[i][k] * (1 - sigmoid(logit))
      if not y[i]:
        dl[k] -= X[i][k]
  for k in range(len(theta)):
    dl[k] -= lam*2*theta[k]
  return numpy.array([-x for x in dl])

In [7]:
##################################################
# Train                                          #
##################################################

def train(lam):
  theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), \
    fprime, pgtol = 10, args = (X_train, y_train, lam))
  return theta
X_data = [X_train, X_validation, X_test]
y_data = [y_train, y_validation, y_test]
symbol = ['train', 'valid', 'test']
print 'λ\tDataset\t\tTruePositive\tFalsePositive\tTrueNegative\tFalseNegative\tAccuracy\tBER'
lam = 1.0
theta = train(lam)
#print theta
for i in range(3):
    def TP(theta):
      scores = [inner(theta,x) for x in X_data[i]]
      predictions = [s > 0 for s in scores]
      correct = [((a==1) and (b==1)) for (a,b) in zip(predictions,y_data[i])]
      tp = sum(correct) * 1.0
      return tp
    def TN(theta):
      scores = [inner(theta,x) for x in X_data[i]]
      predictions = [s > 0 for s in scores]
      correct = [((a==0) and (b==0)) for (a,b) in zip(predictions,y_data[i])]
      tn = sum(correct) * 1.0
      return tn
    def FP(theta):
      scores = [inner(theta,x) for x in X_data[i]]
      predictions = [s > 0 for s in scores]
      correct = [((a==1) and (b==0)) for (a,b) in zip(predictions,y_data[i])]
      fp = sum(correct) * 1.0
      return fp
    def FN(theta):
      scores = [inner(theta,x) for x in X_data[i]]
      predictions = [s > 0 for s in scores]
      correct = [((a==0) and (b==1)) for (a,b) in zip(predictions,y_data[i])]
      fn = sum(correct) * 1.0
      return fn
    if i == 1 or i == 2 :
        tp = TP(theta)
        fp = FP(theta)
        tn = TN(theta)
        fn = FN(theta)
        TPR = tp / (tp + fn)
        TNR = tn / (tn + fp)
        BER = 1 - 0.5 * (TPR + TNR)
        accuracy = (tp+tn)/(tp+tn+fp+fn)
        print str(lam)+'\t'+symbol[i]+'\t\t'+str(tp)+'\t\t'+str(fp)+'\t\t'+str(tn)+'\t\t'+\
              str(fn)+'\t\t'+str(accuracy)+'\t'+str(BER)

λ	Dataset		TruePositive	FalsePositive	TrueNegative	FalseNegative	Accuracy	BER
1.0	valid		14702.0		456.0		302.0		1206.0		0.90027601104	0.338697013102
1.0	test		4910.0		6034.0		4721.0		1003.0		0.577813774898	0.365333811678


In [ ]:
# X_data, y_data can be changed to train, validation or test data
def TP(theta):
  scores = [inner(theta,x) for x in X_data[i]]
  predictions = [s > 0 for s in scores]
  correct = [((a==1) and (b==1)) for (a,b) in zip(predictions,y_data[i])]
  tp = sum(correct) * 1.0
  return tp
def TN(theta):
  scores = [inner(theta,x) for x in X_data[i]]
  predictions = [s > 0 for s in scores]
  correct = [((a==0) and (b==0)) for (a,b) in zip(predictions,y_data[i])]
  tn = sum(correct) * 1.0
  return tn
def FP(theta):
  scores = [inner(theta,x) for x in X_data[i]]
  predictions = [s > 0 for s in scores]
  correct = [((a==1) and (b==0)) for (a,b) in zip(predictions,y_data[i])]
  fp = sum(correct) * 1.0
  return fp
def FN(theta):
  scores = [inner(theta,x) for x in X_data[i]]
  predictions = [s > 0 for s in scores]
  correct = [((a==0) and (b==1)) for (a,b) in zip(predictions,y_data[i])]
  fn = sum(correct) * 1.0
  return fn
tp = TP(theta)
fp = FP(theta)
tn = TN(theta)
fn = FN(theta)
TPR = tp / (tp + fn)
TNR = tn / (tn + fp)
BER = 1 - 0.5 * (TPR + TNR)
accuracy = (tp+tn)/(tp+tn+fp+fn)